Dependencies

In [12]:

import numpy as np

import shapely as shp
import rasterio as rio

In [13]:
work_dir = "./data/"
image = f"{work_dir}LWIR_QuickMosaic_16-bit_9327.tiff"

In [14]:
with rio.open(image) as src:
    band1 = src.read(1)
    profile = src.profile

    for thresh in range(30000, 36000, 1000):

        with rio.open(f"{work_dir}class{thresh}.tif", 'w', **profile) as dst:
            dst.write(np.where(band1 < thresh, 0, 1).astype(rio.uint8), 1)